# Measure Jet multiplicity

In [1]:
import os, sys
sys.path.append(os.getcwd() + "/../")
from prep_ana_II import *
sys.path.append(os.getcwd() + "/../tools/")
sys.path.append(os.getcwd() + "/../tools/jecsys/")
from JECUncertainty import JECUncertainty, jecUncertainties
from lepton_eff import muonEff, electronEff
from cross_section_calc import calc_cross_stuff, cross_calc, stat_unc_calc, normalization_unc_calc

Welcome to JupyROOT 6.10/00
home /home/gunter/WW_analysis/production/Analysis_13TeV
Loading binning options...
Loading plotting options
unc_mc_process and scales are parameter dictionaries


In [2]:
%matplotlib inline

In [3]:
matplotlib.rcParams.update({'font.size': 26})

scales, df = load_testset() 
df_da = load_presel_w_fDY_fTT_DATA()
rfs = load_randomForest()

fDY = rfs["clf_fDY"]
fTT = rfs["clf_fTT"]

features_fDY = rfs["features_fDY"]
features_fTT = rfs["features_fTT"]

##  Est. Measurements

In [63]:
#Number of jets 
def table_jet_multi(df, scales=scales):
    a = []
    for i in range(3):
        query = "pred_fDY_WW > 0.9 & pred_fTT_WW > 0.6 & numb_jets == " + str(i)
        if i == 2:
            query= "pred_fDY_WW > 0.9 & pred_fTT_WW > 0.6 &  numb_jets >= " + str(i)
        a.append(process_yields(pre_cuts(df), df_da, query=query, scales=scales))
    return a
a = table_jet_multi(df)

Skipping Higgs not in data frame


In [64]:
#Construct measurements
def measure_jet_multi(tables):
    total = []
    bkg = []

    for i, ele in enumerate(tables):
        bkg.append(0)
        for process in ele.Process.unique():
            if process == "DATA":
                continue
            if "WW" in process:
                continue 
            if "Total" not in process:
                bkg[i] += ele[ele.Process == process]["Diff Flavor"].values[0] + ele[ele.Process == process]["Same Flavor"].values[0]
            else:
                total.append(ele[ele.Process == process]["Diff Flavor"].values[0] + ele[ele.Process == process]["Same Flavor"].values[0] )
    bkg = np.array(bkg)
    return total, bkg
total, bkg = measure_jet_multi(a)

### Fake data

In [47]:
# construct fake data
fake_data = []
for i in range(3):
    fake_data.append( float(np.random.poisson(total[i],1)[0]) )
fake_data = np.array(fake_data)

### Measurement

In [52]:
# construct measurement
import pickle
matrices = pickle.load(open("../data/Unfolding/june13.pkl", "r"))
sf = matrices["sf"]

fake_measurement = []
for i in range(3):
    fake_measurement.append( sf[i] * float(fake_data[i] - bkg[i]) / ((sf*fake_data).sum() - (sf*bkg).sum())  )
np.array(fake_measurement)
print fake_measurement

[0.66063955989518852, 0.27952391505559981, 0.059836525049211976]


## Unfold Est. 

In [54]:
np.dot(np.linalg.inv(matrices["tot_matrix"]), fake_measurement)

array([ 0.71293043,  0.26083944,  0.02623012])

### nomalization unc

In [68]:
import copy
up = []
down = []
for process in scales:
    if process == "Data":
        continue
    if "WW" in process:
        continue
    if process not in df.process_decay.unique():
        continue
    alt_scales = copy.copy(scales)
    alt_scales[process] = scales[process] * (1. + unc_mc_process[process])
    a = table_jet_multi(df, scales=alt_scales)
    temp_, bkg = measure_jet_multi(a)
    fake_measurement = []
    for i in range(3):
        fake_measurement.append( sf[i] * float(fake_data[i] - bkg[i]) / ((sf*fake_data).sum() - (sf*bkg).sum())  )
    up.append(np.array(fake_measurement))
    
    alt_scales = copy.copy(scales)
    alt_scales[process] = scales[process] * (1. - unc_mc_process[process])
    a = table_jet_multi(df, scales=alt_scales)
    temp_, bkg = measure_jet_multi(a)
    fake_measurement = []
    for i in range(3):
        fake_measurement.append( sf[i] * float(fake_data[i] - bkg[i]) / ((sf*fake_data).sum() - (sf*bkg).sum())  )
    down.append(np.array(fake_measurement))
up = np.array(up)
down = np.array(down)

Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping Higgs not in data frame
Skipping H

In [76]:
norm_unc =  ((np.abs( up - down ) / 2.)**2.).sum(0)**.5
print norm_unc

[ 0.00774534  0.00510496  0.00299589]


### Stat